# 데이터 가져오기
- 데이터 참조 : https://catalog.data.gov/dataset/?tags=taxis
- 총 다운로드 시간 5-6시간

In [9]:
import polars as pl
pl.show_versions()

--------Version info---------
Polars:              1.22.0
Index type:          UInt32
Platform:            Windows-11-10.0.22621-SP0
Python:              3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
LTS CPU:             False

----Optional dependencies----
Azure CLI            <not installed>
adbc_driver_manager  <not installed>
altair               <not installed>
azure.identity       <not installed>
boto3                <not installed>
cloudpickle          <not installed>
connectorx           <not installed>
deltalake            0.24.0
fastexcel            <not installed>
fsspec               <not installed>
gevent               <not installed>
google.auth          <not installed>
great_tables         <not installed>
matplotlib           3.9.4
numpy                2.0.2
openpyxl             3.1.5
pandas               2.2.3
pyarrow              19.0.0
pydantic             <not installed>
pyiceberg            <not installed>
sqlalchemy           <not

In [10]:
import os

file_path = 'data/ch05/creditcard.csv'
file_size = os.path.getsize(file_path)

def convert_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0

print(f"File size: {convert_size(file_size)}")

File size: 143.84 MB


# 레시피 24 : 데이터 수집 속도 비교

In [11]:
import pandas as pd
import polars as pl
import time

file_path = 'data/ch05/creditcard.csv'

# Pandas 데이터 로드 시간 측정
start_time = time.time()
df_pandas = pd.read_csv(file_path)
pandas_time = time.time() - start_time
print(f"Pandas 로드 시간: {pandas_time:.2f}초")

# Polars (Lazy API + Streaming) 데이터 로드 시간 측정
start_time = time.time()
# Time 컬럼을 Float64로 지정
df_polars_lazy = pl.scan_csv(file_path, schema_overrides={"Time": pl.Float64}).collect()  
polars_lazy_time = time.time() - start_time
print(f"Polars (Lazy API + Streaming Mode) 로드 시간: {polars_lazy_time:.2f}초")

Pandas 로드 시간: 1.24초
Polars (Lazy API + Streaming Mode) 로드 시간: 0.12초


# 레시피 25 : 메모리 사용 측정

In [9]:
import pandas as pd 
import polars as pl
import psutil
import os
import gc

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

def convert_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0

# 파일 경로
file_path = 'data/ch05/creditcard.csv'

# 초기 메모리 사용량 측정
initial_memory = get_memory_usage()

# Pandas 메모리 사용량 측정
df_pandas = pd.read_csv(file_path)
pandas_memory = get_memory_usage() - initial_memory
print(df_pandas.head(1))

# 메모리 정리
del df_pandas
_ = gc.collect()

# 다시 초기 메모리 측정
initial_memory = get_memory_usage()

# Polars Lazy 메모리 사용량 측정
df_polars_lazy = pl.scan_csv(file_path, schema_overrides={"Time": pl.Float64})
polars_memory = get_memory_usage() - initial_memory

print(df_polars_lazy.head(1).collect())

del df_polars_lazy
_ = gc.collect()

print(f"Pandas 메모리 사용량: {convert_size(pandas_memory)}")
print(f"Polars Lazy 메모리 사용량: {convert_size(polars_memory)}")
print(f"메모리 사용량 차이: {convert_size(abs(pandas_memory - polars_memory))}")

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  

[1 rows x 31 columns]
shape: (1, 31)
┌──────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬────────┬───────┐
│ Time ┆ V1        ┆ V2        ┆ V3       ┆ … ┆ V27      ┆ V28       ┆ Amount ┆ Class │
│ ---  ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---    ┆ ---   │
│ f64  ┆ f64       ┆ f64       ┆ f64      ┆   ┆ f64      ┆ f64       ┆ f64    ┆ i64   │
╞══════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪════════╪═══════╡
│ 0.0  ┆ -1.359807 ┆ -0.072781 ┆ 2.536347 ┆ … ┆ 0.133558 ┆ -0.021053 ┆ 149.62 ┆ 0     │
└──────┴────